In [ ]:
from datasets import load_dataset
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
from unimemrag.embedding.models.ClipEmbedding import ClipEmbedding
from unimemrag.vector_store.qdrant import QdrantStore
from config import Config

In [ ]:
import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

cfg = Config(collection="wit", prefer_grpc=False)
embed_model = ClipEmbedding(cfg.model_name)
vector_store = QdrantStore(cfg, vector_size=embed_model.dim)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("google/wit", split="train", streaming=True)

In [1]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib
import aiohttp
import PIL.Image

from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent

import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"


def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": get_datasets_user_agent()},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image


def fetch_images(batch, num_threads, timeout=None, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch


num_threads = 20
dset = load_dataset("google/wit", split="train[:1%]", storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=36000)}})
# dset = dset.map(fetch_images, batched=True, batch_size=1000, fn_kwargs={"num_threads": num_threads})

/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dset[1]

{'language': 'cs',
 'page_url': 'https://cs.wikipedia.org/wiki/Mechanick%C3%A1_m%C3%AD%C5%99idla',
 'image_url': 'http://upload.wikimedia.org/wikipedia/commons/2/2b/M16_rifle_correct_sight_picture_fig_4-18.png',
 'page_title': 'Mechanická mířidla',
 'section_title': None,
 'hierarchical_section_title': 'Mechanická mířidla',
 'caption_reference_description': 'Mechanická mířidla na útočné pušce M16',
 'caption_attribution_description': 'M16 rifle correct sight picture',
 'caption_alt_text_description': None,
 'mime_type': 'image/png',
 'original_height': 606,
 'original_width': 1182,
 'is_main_image': True,
 'attribution_passes_lang_id': True,
 'page_changed_recently': True,
 'context_page_description': 'Mechanická mířidla je zařízení určené pro zamíření střelné zbraně na zvolený cíl. V některých pramenech se používá označení mechanické zaměřovače. Vedle mechanických existují ještě optické zaměřovače. Mechanické zaměřovače se dále dělí na otevřené a dioptrové.',
 'context_section_descrip